In [ ]:
# Copyright 2022 Piotr Szukalski

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Visualizing Solar Bodies Movement

Code developed for [It's There Blog](https://blog.itsthere.academy/) - "I am an astronomer!"

1. Check the requirements.txt for packages
2. Download 2 files - so called 'kernels' from SPICE website. Place them in 'data' directory
    1. de430.bsp - planets ephemerids from [here](https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/)
    2. naif0012.tls - leap seconds from [here](https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/)

In [ ]:
import spiceypy as spice
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, UTC
import pandas as pd
import plotly.express as px

In [ ]:
METAKR = 'getsta.tm'

spice.tkvrsn('TOOLKIT')
spice.unload(METAKR)

# The crucial bit - load the kernels (data files)
spice.furnsh(METAKR)


def gen_pos(object, observer, ref, ts):
    """
    Generate x, y, z coordinates (ephemerids) for given time frame.

    :param object: the body to generate the ephemerids for
    :param observer: object observing the movement, or relative to what body the object moves?
    :param ref: reference frame, or coordinates system (e.g. "ECLIPJ2000" for the ecliptic)
    :param ts: timestamp

    :return: ndarray of (x, y, z) coodrindates
    """

    ts_bt = spice.datetime2et(ts)   
    # spkpos returns two vectors - position and velocity. We care the position only for now.
    (pos_t, _) = spice.spkpos(object, ts_bt, ref, 'NONE', observer)
    
    return pos_t


In [ ]:
PLANET_TO_SPICE = {
    'SUN': 'SUN',
    'MERCURY': 'MERCURY',
    'VENUS': 'VENUS',
    'EARTH': 'EARTH',
    'MARS': 'MARS BARYCENTER',
    'JUPITER': 'JUPITER BARYCENTER',
    'SATURN': 'SATURN BARYCENTER',
    'URANUS': 'URANUS BARYCENTER',
    'NEPTUNE': 'NEPTUNE BARYCENTER',
    'PLUTO': 'PLUTO BARYCENTER'
}

planet_radii_km = {
    'SUN': 696340,
    'MERCURY': 2439.7,
    'VENUS': 6051.8,
    'EARTH': 6371.0,
    'MARS': 3389.5,
    'JUPITER': 69911,
    'SATURN': 58232,
    'URANUS': 25362,
    'NEPTUNE': 24622,
    'PLUTO': 1188.3
}

now = datetime.now(UTC)

start_date = now - pd.DateOffset(months=6)
end_date = now + pd.DateOffset(months=6)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

bodies_pos = []

for body, body_spice in PLANET_TO_SPICE.items():
    for ts in date_range:
        ts_midnight = ts.replace(hour=0, minute=0, second=0, microsecond=0)
        pos = gen_pos(body_spice, 'SUN', 'ECLIPJ2000', ts_midnight)
        bodies_pos.append({'body': body, 'date': ts_midnight.date(), 'x': pos[0], 'y': pos[1], 'z': pos[2]})

bodies_df = pd.DataFrame(bodies_pos)
planet_radii_df = pd.DataFrame(list(planet_radii_km.items()), columns=['body', 'radius'])
bodies_df = pd.merge(bodies_df, planet_radii_df, on='body')

bodies_df

In [ ]:
def plot_solar_bodies(bodies_df):
    # Calculate the range for the axes
    range_val = np.max(np.abs(bodies_df[['x', 'y']])) * 1.1

    # Create the color dictionary
    color_dict = {row['body']: ('orange' if row['body'] == 'SUN' else 'black') for index, row in bodies_df.iterrows()}

    # Create the scatter plot
    fig = px.scatter(bodies_df, x='x', y='y', text='body', 
                     color='body',
                     color_discrete_map=color_dict,
                     animation_frame="date", animation_group="body")

    # Update marker properties
    fig.update_traces(marker=dict(
        size=bodies_df['body'].apply(lambda x: 7 if x == 'SUN' else 1)
    ))

    # Update layout for background color, grid lines, and aspect ratio
    fig.update_layout(
        # plot_bgcolor='black',
        # paper_bgcolor='black',
        xaxis=dict(
            gridcolor='grey',
            gridwidth=1,
            griddash='dash'
        ),
        yaxis=dict(
            gridcolor='grey',
            gridwidth=1,
            griddash='dash',
            scaleanchor="x",
            scaleratio=1
        ),
        width=800,
        height=800,
        showlegend=False
    )

    # Update axes range
    fig.update_xaxes(range=[-range_val, range_val])
    fig.update_yaxes(range=[-range_val, range_val])

    fig.show()

    return fig

### All planets

In [ ]:
all_planets_fig = plot_solar_bodies(bodies_df)
#all_planets_fig.write_html('/tmp/all_planets.html', include_plotlyjs='cdn', include_mathjax='cdn', full_html=False)

### Inner Solar System

In [ ]:
inner_solar_fig = plot_solar_bodies(bodies_df[bodies_df['body'].isin(['SUN', 'MERCURY', 'VENUS', 'EARTH', 'MARS'])])
#inner_solar_fig.write_html('/tmp/inner_solar.html', include_plotlyjs='cdn', include_mathjax='cdn', full_html=False)

### Outer Solar System + Sun and Earth

In [ ]:
outer_with_earth_fig = plot_solar_bodies(bodies_df[bodies_df['body'].isin(['SUN', 'EARTH', 'JUPITER', 'SATURN', 'URANUS', 'NEPTUNE', 'PLUTO'])])
#outer_with_earth_fig.write_html('/tmp/outer_with_earth.html', include_plotlyjs='cdn', include_mathjax='cdn', full_html=False)